# PM 5 Task H POD Truncation - match Demircan


In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
from scipy.sparse.linalg import eigs
import time

## Full Order Model

In [ ]:
FIRST_RUN = False # True saves outputs, False loads outputs
LEAPFROG = False # True runs leapfrog, False runs trapezoidal
SAME_SOURCE = True # True compares to full order solve

In [ ]:
from setup_sonar_model import setup_sonar_model, print_model_info
from eval_u_Sonar import eval_u_Sonar_20, eval_u_Sonar_20_const

if SAME_SOURCE:
    eval_u = eval_u_Sonar_20_const
else:
    eval_u = eval_u_Sonar_20

# Setup model
model = setup_sonar_model(
    Nx=1001,
    Nz=251,
    Lx=2000,
    Lz=500,
    f0=20,
    source_position="center",
    hydrophone_config="horizontal",
    eval_u = eval_u,
)

# Print info
print_model_info(model)

# Extract what you need
p = model['p']
x_start = model['x_start']
t_sim = model['t_stop']
max_dt_FE = model['max_dt_FE']
eval_u_scaled = model['eval_u_scaled']
eval_f = model['eval_f']
f0 = model['f0']

A = p['A']
B = p['B']

Nx, Nz = model['Nx'], model['Nz']
Lx, Lz = model['Lx'], model['Lz']
N = Nx * Nz
n = 2 * N

print(f"\nState dimension: 2N = {n:,}")

# Build C matrix for hydrophone outputs
hydro = model['hydrophones']
n_phones = hydro['n_phones']

if 'z_pos' in hydro:
    z_idx = hydro['z_pos']
    x_indices = hydro['x_indices']
    C = np.zeros((n_phones, n))
    for i, x_idx in enumerate(x_indices):
        obs_idx = N + x_idx * Nz + z_idx
        C[i, obs_idx] = 1.0
else:
    x_indices = hydro['x_indices']
    z_indices = hydro['z_indices']
    C = np.zeros((n_phones, n))
    for i, (x_idx, z_idx) in enumerate(zip(x_indices, z_indices)):
        obs_idx = N + x_idx * Nz + z_idx
        C[i, obs_idx] = 1.0

### Input function

In [ ]:
# Visualize pulse
t_test = np.linspace(0, t_sim, 1000)   
u_test = [eval_u_scaled(t) for t in t_test]

plt.figure(figsize=(10, 3))
plt.plot(t_test * 1000, u_test, 'b-', linewidth=1.5)
plt.xlabel('Time (ms)')
plt.ylabel('u(t)')
plt.title(f'Input Pulse: {f0} Hz Gaussian-windowed sine')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

Full model simulation

In [ ]:
from scipy.sparse import eye, csr_matrix
from scipy.sparse.linalg import splu

if FIRST_RUN:
    # Timestep
    dt = max_dt_FE * 0.5  # Can use larger dt with trapezoidal (A-stable)
    num_steps = int(np.ceil(t_sim / dt))

    print(f"\nRunning full-order TRAPEZOIDAL simulation...")
    print(f"  Duration: {t_sim*1000:.0f} ms")
    print(f"  Timestep: {dt*1e6:.1f} μs")
    print(f"  Steps: {num_steps:,}")
    print(f"  DOFs: {n:,}")

    # Precompute LU factorization (this is the expensive part)
    print("  Computing LU factorization of (I - dt/2 * A)...")
    t0_lu = time.perf_counter()
    
    I_sparse = eye(n, format='csr')
    LHS = I_sparse - (dt/2) * A  # (I - dt/2 * A)
    RHS_mat = I_sparse + (dt/2) * A  # (I + dt/2 * A)
    
    # Sparse LU factorization
    LU = splu(LHS.tocsc())
    
    lu_time = time.perf_counter() - t0_lu
    print(f"  LU factorization done in {lu_time:.1f}s")

    # Allocate storage
    X_full = np.zeros((n, num_steps + 1))
    t_full = np.zeros(num_steps + 1)
    
    x_curr = x_start.flatten().copy()
    B_dense = B.toarray().flatten()
    
    X_full[:, 0] = x_curr
    t_full[0] = 0.0

    # Time integration
    print("  Running time integration...")
    t0 = time.perf_counter()
    
    for i in range(1, num_steps + 1):
        if i % max(1, num_steps // 10) == 0:
            print(f"    Progress: {100*i/num_steps:.0f}%")
        
        t_prev = t_full[i-1]
        t_curr = t_prev + dt
        
        u_prev = eval_u_scaled(t_prev)
        u_curr = eval_u_scaled(t_curr)
        
        # Trapezoidal: (I - dt/2*A) x_{n+1} = (I + dt/2*A) x_n + dt/2 * B * (u_n + u_{n+1})
        rhs = RHS_mat @ x_curr + (dt/2) * B_dense * (u_prev + u_curr)
        x_curr = LU.solve(rhs)
        
        X_full[:, i] = x_curr
        t_full[i] = t_curr
    
    full_time = time.perf_counter() - t0
    print(f"\n✓ Full-order complete in {full_time:.1f}s (+ {lu_time:.1f}s LU)")
    print(f"  Snapshot matrix: {X_full.shape}")

    # Verify BC
    surface_p = X_full[N::Nz, :]
    print(f"  Surface pressure max: {np.abs(surface_p).max():.2e} (should be ~0)")

    # Extract hydrophone outputs
    y_full = (C @ X_full).T
    print(f"  Hydrophone outputs: {y_full.shape}")

    # Save
    print("Saving full-order simulation...")
    np.savez_compressed('pod_outputs/full_order_FINAL_trap.npz',
        X_full=X_full,
        t_full=t_full,
        y_full=y_full,
        full_time=full_time,
        Nx=Nx, Nz=Nz, Lx=Lx, Lz=Lz,
        dt=dt, t_sim=t_sim
    )
    print(f"✓ Saved to full_order_FINAL_trap.npz")

else:
    # Load
    data = np.load('pod_outputs/full_order_FINAL_trap.npz')
    X_full = data['X_full']
    t_full = data['t_full']
    y_full = data['y_full']
    full_time = float(data['full_time'])
    dt = float(data['dt'])
    print(f"Loaded X_full: {X_full.shape}")
    print(f"Original solve time: {full_time:.1f}s")

Compute POD basis

In [ ]:
from sklearn.decomposition import TruncatedSVD

if FIRST_RUN:
    # Subsample
    skip = 10
    X_sub = X_full[:, ::skip]
    print(f"Subsampled: {X_sub.shape}")

    k = 200

    print(f"\nComputing randomized SVD (k={k})...")
    t0 = time.perf_counter()

    svd = TruncatedSVD(n_components=k, algorithm='randomized', n_iter=5, random_state=42)
    svd.fit(X_sub.T)  # Transposed!

    U = svd.components_.T  # (n_states, k)
    S = svd.singular_values_

    svd_time = time.perf_counter() - t0
    print(f"✓ Done in {svd_time:.1f}s")

    np.savez_compressed('pod_outputs/svd_FINAL_trap.npz',
        U=U, S=S, k=k,
        svd_time=svd_time, 
        skip=skip,
        Nx=Nx, Nz=Nz, Lx=Lx, Lz=Lz
    )
    print("✓ Saved SVD")
    
    # Energy analysis
    cumulative_energy = np.cumsum(S**2) / np.sum(S**2)
    print(f"Top {k} modes energy: {cumulative_energy[-1]*100:.2f}%")

else:
    data = np.load('pod_outputs/svd_FINAL_trap.npz')
    U = data['U']
    S = data['S']
    k = int(data['k'])
    Nx, Nz = int(data['Nx']), int(data['Nz'])
    Lx, Lz = float(data['Lx']), float(data['Lz'])
    svd_time = data['svd_time']
    data.close()

    print(f"Loaded U: {U.shape}, S: {S.shape}")
    print(f"Grid: {Nx} x {Nz}")
    print(svd_time)

    # Energy analysis
    cumulative_energy = np.cumsum(S**2) / np.sum(S**2)
    print(f"Top {k} modes energy: {cumulative_energy[-1]*100:.2f}%")

POD energy analysis

In [ ]:
cumulative_energy = np.cumsum(S**2) / np.sum(S**2)
n_90 = np.searchsorted(cumulative_energy, 0.90) + 1
n_99 = np.searchsorted(cumulative_energy, 0.99) + 1
n_999 = np.searchsorted(cumulative_energy, 0.999) + 1

print(f"Energy thresholds:")
print(f"  90% energy:   {n_90} modes")
print(f"  99% energy:   {n_99} modes")
print(f"  99.9% energy: {n_999} modes")

# Plot singular value decay
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.semilogy(S, 'b.-', markersize=3)
ax1.set_xlabel('Mode index')
ax1.set_ylabel('Singular value')
ax1.set_title('Singular value decay')
ax1.grid(True, alpha=0.3)

ax2.plot(cumulative_energy * 100, 'b.-', markersize=3)
ax2.axhline(90, color='r', linestyle='--', label='90%')
ax2.axhline(99, color='g', linestyle='--', label='99%')
ax2.axhline(99.9, color='orange', linestyle='--', label='99.9%')
ax2.set_xlabel('Number of modes')
ax2.set_ylabel('Cumulative energy (%)')
ax2.legend()
ax2.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

Stability check on q values

In [ ]:
q_values = list(range(1, 201, 1))  

results = []

print(f"{'q':<6} {'Max Re(λ)':<15} {'Status'}")
print("-" * 40)

for q in q_values:
    if q > U.shape[1]:
        break
    
    Phi_q = U[:, :q]
    A_pod_q = Phi_q.T @ (A @ Phi_q)
    
    eigs = np.linalg.eigvals(A_pod_q)
    max_re = np.real(eigs).max()
    
    # Stable if max Re(λ) ≤ 0
    stable = max_re <= 0
    status = "✓ Stable" if stable else "✗ Unstable"
    
    results.append({
        'q': q,
        'max_re': max_re,
        'stable': stable
    })
    
    print(f"{q:<6} {max_re:<15.4e} {status}")


qs = [r['q'] for r in results]
max_res = [r['max_re'] for r in results]
stable = [r['stable'] for r in results]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Max Re(λ) vs q
colors = ['green' if s else 'red' for s in stable]
axes[0].scatter(qs, max_res, c=colors, s=50)
axes[0].axhline(0, color='k', linestyle='--', lw=1, label='Stability boundary (Re(λ)=0)')
axes[0].set_xlabel('Number of modes (q)')
axes[0].set_ylabel('Max Re(λ)')
axes[0].set_title('ROM Eigenvalues vs q (green=stable, red=unstable)')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Binary stability map
axes[1].bar(qs, [1 if s else 0 for s in stable], 
            color=['green' if s else 'red' for s in stable], width=4, alpha=0.7)
axes[1].set_xlabel('Number of modes (q)')
axes[1].set_ylabel('Stable (1) / Unstable (0)')
axes[1].set_title('Stability Map (Re(λ) ≤ 0)')
axes[1].set_ylim([0, 1.2])
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary
stable_qs = [r['q'] for r in results if r['stable']]
unstable_qs = [r['q'] for r in results if not r['stable']]

print("\n" + "="*60)
print("STABILITY SUMMARY (Re(λ) ≤ 0)")
print("="*60)
print(f"Stable q values: {stable_qs}")
print(f"Unstable q values: {unstable_qs}")
if stable_qs:
    print(f"\nRecommended: q={max(stable_qs)} (highest stable)")
print("="*60)

Build POD ROM - sparse projection (don't convert A to dense)

In [ ]:
#q_pod = n_999
q_pod = max(stable_qs)
print(f"Building POD-ROM with q={q_pod} modes ({cumulative_energy[q_pod-1]*100:.2f}% energy)")

Phi = U[:, :q_pod]

# Sparse projection - A stays sparse!
print("Projecting A (sparse)...")
A_Phi = A @ Phi           # (n × q) sparse @ dense = dense
A_pod = Phi.T @ A_Phi     # (q × n) @ (n × q) = (q × q)

B_pod = Phi.T @ B.toarray()
C_pod = C @ Phi
x0_pod = Phi.T @ x_start  # <-- Add this line

N = Nx * Nz
n = 2 * N

print(f"\nA_pod: {A_pod.shape}")
print(f"B_pod: {B_pod.shape}")
print(f"C_pod: {C_pod.shape}")
print(f"Compression: {100*q_pod/n:.4f}%")

# Check stability
eigs_pod = np.linalg.eigvals(A_pod)
max_re = np.real(eigs_pod).max()
print(f"\nMax Re(λ): {max_re:.2e} (should be ≤ 0)")
if max_re > 0:
    print(f"⚠ Unstable - growth factor over {t_sim:.1f}s: {np.exp(max_re * t_sim):.2f}x")
else:
    print("✓ Stable")

Simulate POD ROM and compare

In [ ]:
def simulate_pod_leapfrog(A_pod, B_pod, C_pod, u_func, t_span, dt, x0=None):
    """Leapfrog integrator for POD-ROM"""
    t = np.arange(t_span[0], t_span[1] + dt, dt)
    n_steps = len(t)
    q = A_pod.shape[0]
    n_out = C_pod.shape[0]
    
    x = np.zeros((n_steps, q))
    y = np.zeros((n_steps, n_out))
    
    x_prev = x0.flatten().copy() if x0 is not None else np.zeros(q)
    x_curr = x_prev.copy()
    
    # RK4 bootstrap
    u0 = u_func(t[0])
    k1 = A_pod @ x_curr + B_pod.flatten() * u0
    k2 = A_pod @ (x_curr + 0.5*dt*k1) + B_pod.flatten() * u_func(t[0] + 0.5*dt)
    k3 = A_pod @ (x_curr + 0.5*dt*k2) + B_pod.flatten() * u_func(t[0] + 0.5*dt)
    k4 = A_pod @ (x_curr + dt*k3) + B_pod.flatten() * u_func(t[0] + dt)
    x_next = x_curr + (dt/6) * (k1 + 2*k2 + 2*k3 + k4)
    
    x[0], y[0] = x_curr, C_pod @ x_curr
    x_prev, x_curr = x_curr.copy(), x_next.copy()
    
    for i in range(1, n_steps):
        x[i], y[i] = x_curr, C_pod @ x_curr
        if i < n_steps - 1:
            f_i = A_pod @ x_curr + B_pod.flatten() * u_func(t[i])
            x_next = x_prev + 2 * dt * f_i
            x_prev, x_curr = x_curr.copy(), x_next.copy()
    
    return t, y, x


def simulate_pod_trapezoidal(A_pod, B_pod, C_pod, u_func, t_span, dt, x0=None):
    """Trapezoidal (Crank-Nicolson) integrator - A-stable for any Re(λ) ≤ 0"""
    t = np.arange(t_span[0], t_span[1] + dt, dt)
    n_steps = len(t)
    q = A_pod.shape[0]
    n_out = C_pod.shape[0]
    
    x = np.zeros((n_steps, q))
    y = np.zeros((n_steps, n_out))
    
    x_curr = x0.flatten().copy() if x0 is not None else np.zeros(q)
    B_flat = B_pod.flatten()
    
    # Precompute matrices for trapezoidal rule:
    # x_{n+1} = x_n + (dt/2) * (f_n + f_{n+1})
    # x_{n+1} = x_n + (dt/2) * (A x_n + B u_n + A x_{n+1} + B u_{n+1})
    # (I - dt/2 * A) x_{n+1} = (I + dt/2 * A) x_n + (dt/2) * B * (u_n + u_{n+1})
    
    I = np.eye(q)
    LHS = I - (dt/2) * A_pod  # Left-hand side matrix
    RHS_mat = I + (dt/2) * A_pod  # Right-hand side matrix
    
    # Precompute LU factorization for efficiency
    from scipy.linalg import lu_factor, lu_solve
    LU = lu_factor(LHS)
    
    x[0] = x_curr
    y[0] = C_pod @ x_curr
    
    for i in range(1, n_steps):
        u_prev = u_func(t[i-1])
        u_curr = u_func(t[i])
        
        rhs = RHS_mat @ x_curr + (dt/2) * B_flat * (u_prev + u_curr)
        x_curr = lu_solve(LU, rhs)
        
        x[i] = x_curr
        y[i] = C_pod @ x_curr
    
    return t, y, x

In [ ]:
if LEAPFROG:
    # Run POD-ROM
    x0_pod = Phi.T @ x_start

    t0 = time.perf_counter()
    t_pod, y_pod, x_pod = simulate_pod_leapfrog(A_pod, B_pod, C_pod, eval_u_scaled, (0, t_sim), dt, x0_pod)
    pod_time = time.perf_counter() - t0

    print(f"✓ POD-ROM complete in {pod_time:.3f}s")
    print(f"  Speedup: {full_time/pod_time:.0f}x")

else:
    print(f"Testing q={q_pod} with Trapezoidal...")
    t0 = time.perf_counter()
    t_pod, y_pod, x_pod = simulate_pod_trapezoidal(A_pod, B_pod, C_pod, eval_u_scaled, (0, t_sim), dt, x0_pod)
    pod_time = time.perf_counter() - t0

    print(f"\nTrapezoidal time: {pod_time:.3f}s")
    print(f"Full-order time:  {full_time:.1f}s")
    print(f"Speedup:          {full_time/pod_time:.0f}x")
    print(f"\nMax |y_pod|:  {np.abs(y_pod).max():.2e}")
    print(f"Max |y_full|: {np.abs(y_full).max():.2e}")

    if SAME_SOURCE:
        # Compute errors
        errors = []
        for i in range(n_phones):
            y_pod_interp = np.interp(t_full, t_pod, y_pod[:, i])
            rel_err = np.linalg.norm(y_full[:, i] - y_pod_interp) / (np.linalg.norm(y_full[:, i]) + 1e-12)
            errors.append(rel_err)

        print(f"\nHydrophone errors:")
        for i, err in enumerate(errors):
            print(f"  H{i+1}: {100*err:.2f}%")
        print(f"Average error: {100*np.mean(errors):.2f}%")

In [ ]:
if SAME_SOURCE:
    # Hydrophone comparison
    errors = []

    fig, axes = plt.subplots(n_phones, 2, figsize=(14, 3*n_phones))
    if n_phones == 1:
        axes = axes.reshape(1, -1)

    for i in range(n_phones):
        y_pod_interp = np.interp(t_full, t_pod, y_pod[:, i])
        err = y_full[:, i] - y_pod_interp
        rel_err = np.linalg.norm(err) / (np.linalg.norm(y_full[:, i]) + 1e-12)
        errors.append(rel_err)
        
        axes[i, 0].plot(t_full*1000, y_full[:, i], 'b-', lw=1.5, label='Full', alpha=0.7)
        axes[i, 0].plot(t_pod*1000, y_pod[:, i], 'r--', lw=1.5, label=f'POD (q={q_pod})')
        axes[i, 0].set_ylabel('Pressure (Pa)')
        axes[i, 0].set_title(f'H{i+1}: Full vs POD')
        axes[i, 0].legend()
        axes[i, 0].grid(True, alpha=0.3)
        
        axes[i, 1].plot(t_full*1000, err, 'g-', lw=1)
        axes[i, 1].set_ylabel('Error (Pa)')
        axes[i, 1].set_title(f'H{i+1} Error (rel: {100*rel_err:.2f}%)')
        axes[i, 1].grid(True, alpha=0.3)

    axes[-1, 0].set_xlabel('Time (ms)')
    axes[-1, 1].set_xlabel('Time (ms)')
    plt.tight_layout()
    plt.show()

    # Summary
    print("=" * 60)
    print("POD-ROM RESULTS")
    print("=" * 60)
    print(f"Full-order: {n:,} DOFs, {full_time:.1f}s")
    print(f"POD-ROM:    {q_pod} DOFs, {pod_time:.3f}s")
    print(f"Speedup:    {full_time/pod_time:.0f}x")
    print(f"\nRelative errors:")
    for i, err in enumerate(errors):
        print(f"  H{i+1}: {100*err:.4f}%")
    print("=" * 60)

In [ ]:
'''
# Reconstruct full state from POD
X_pod_reconstructed = Phi @ x_pod.T

# Check surface pressure (should be ~0 for p=0 BC)
N = Nx * Nz
surface_p_full = X_full[N::Nz, :]      # Every Nz-th pressure node (j=0)
surface_p_pod = X_pod_reconstructed[N::Nz, :]

print("Surface pressure (p=0 at z=0):")
print(f"  Full-order max: {np.abs(surface_p_full).max():.2e}")
print(f"  POD-ROM max:    {np.abs(surface_p_pod).max():.2e}")

# Vertical array visualization
def extract_vertical_array(p, X, x_pos=None):
    Nx, Nz = p['Nx'], p['Nz']
    N = Nx * Nz
    if x_pos is None:
        x_pos = Nx // 2
    z_indices = list(range(0, Nz))  # Include surface!
    pressure = np.zeros((len(z_indices), X.shape[1]))
    for di, zi in enumerate(z_indices):
        flat_idx = x_pos * Nz + zi
        pressure[di, :] = X[N + flat_idx, :]
    depths = np.array(z_indices) * p['dz']
    return pressure, depths

# Extract from both
pressure_full, depths = extract_vertical_array(p, X_full)
pressure_pod, _ = extract_vertical_array(p, X_pod_reconstructed)

# Match lengths
n_times = min(pressure_full.shape[1], pressure_pod.shape[1])
pressure_full = pressure_full[:, :n_times]
pressure_pod = pressure_pod[:, :n_times]
t_plot = t_full[:n_times]

t_ms = np.array(t_plot) * 1000
vmax = max(np.abs(pressure_full).max(), np.abs(pressure_pod).max())

fig, axes = plt.subplots(1, 3, figsize=(16, 6))

# Full-order
im0 = axes[0].imshow(pressure_full, aspect='auto', cmap='RdBu_r',
                      extent=[t_ms[0], t_ms[-1], depths[-1], depths[0]],
                      vmin=-vmax, vmax=vmax)
axes[0].axhline(p['sonar_iz'] * p['dz'], color='yellow', linestyle='--', lw=2, label='Source')
axes[0].axhline(0, color='cyan', linestyle='-', lw=2, label='Surface (p=0)')
axes[0].axhline(p['Lz'], color='brown', linestyle='-', lw=2, label='Seafloor')
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Depth (m)')
axes[0].set_title(f'Full-order ({full_time:.1f}s)')
axes[0].legend(loc='upper right', fontsize=8)
plt.colorbar(im0, ax=axes[0], label='Pressure (Pa)')

# POD-ROM
im1 = axes[1].imshow(pressure_pod, aspect='auto', cmap='RdBu_r',
                      extent=[t_ms[0], t_ms[-1], depths[-1], depths[0]],
                      vmin=-vmax, vmax=vmax)
axes[1].axhline(p['sonar_iz'] * p['dz'], color='yellow', linestyle='--', lw=2)
axes[1].axhline(0, color='cyan', linestyle='-', lw=2)
axes[1].axhline(p['Lz'], color='brown', linestyle='-', lw=2)
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('Depth (m)')
axes[1].set_title(f'POD-ROM q={q_pod} ({pod_time:.2f}s)')
plt.colorbar(im1, ax=axes[1], label='Pressure (Pa)')

# Error
error = pressure_pod - pressure_full
err_max = np.abs(error).max()
im2 = axes[2].imshow(error, aspect='auto', cmap='RdBu_r',
                      extent=[t_ms[0], t_ms[-1], depths[-1], depths[0]],
                      vmin=-err_max, vmax=err_max)
axes[2].set_xlabel('Time (ms)')
axes[2].set_ylabel('Depth (m)')
rel_err = np.linalg.norm(error) / (np.linalg.norm(pressure_full) + 1e-12) * 100
axes[2].set_title(f'Error (rel: {rel_err:.2f}%)')
plt.colorbar(im2, ax=axes[2], label='Error (Pa)')

plt.suptitle('Boundary Condition Verification: Full vs POD-ROM', fontsize=12)
plt.tight_layout()
plt.show()

# Surface time series comparison
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(t_ms, pressure_full[0, :], 'b-', lw=1.5, label='Full-order (surface)')
ax.plot(t_ms, pressure_pod[0, :], 'r--', lw=1.5, label='POD-ROM (surface)')
ax.axhline(0, color='k', linestyle=':', alpha=0.5)
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Pressure at surface (Pa)')
ax.set_title('Surface BC check: p(z=0) should be ≈ 0')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nSurface BC violation (RMS):")
print(f"  Full-order: {np.sqrt(np.mean(pressure_full[0, :]**2)):.2e} Pa")
print(f"  POD-ROM:    {np.sqrt(np.mean(pressure_pod[0, :]**2)):.2e} Pa")
'''

In [ ]:
if SAME_SOURCE:
    %matplotlib qt

    import matplotlib.pyplot as plt
    from matplotlib.widgets import Slider, Button
    import numpy as np

    # Select frames
    n_frames = 200
    frame_indices = np.linspace(0, (X_full.shape[1] - 1)/2, n_frames, dtype=int)

    N = Nx * Nz

    # Get global colorscale
    vmax_global = 0
    for idx in frame_indices[::10]:
        p_sample = X_full[N:, idx]
        vmax_global = max(vmax_global, np.abs(p_sample).max())

    vmax_plot = vmax_global * 0.1
    err_max = vmax_plot * 0.01

    # Figure sizing
    domain_aspect = Lx / Lz
    plot_width = 12
    plot_height = plot_width / domain_aspect
    fig_height = 3 * plot_height + 2.5  # Extra space for slider

    # Create figure
    fig, axes = plt.subplots(3, 1, figsize=(plot_width + 2, fig_height))
    plt.subplots_adjust(bottom=0.15, hspace=0.25)

    # Initialize plots
    p_full = X_full[N:, 0].reshape(Nx, Nz).T
    p_pod = (Phi @ x_pod[0, :])[N:].reshape(Nx, Nz).T
    p_error = p_full - p_pod

    im0 = axes[0].imshow(p_full, aspect='equal', cmap='RdBu_r', vmin=-vmax_plot, vmax=vmax_plot,
                        extent=[0, Lx, Lz, 0])
    axes[0].set_ylabel('Depth (m)')
    plt.colorbar(im0, ax=axes[0], label='Pa', fraction=0.046, pad=0.04)
    title0 = axes[0].set_title('Full-order')

    im1 = axes[1].imshow(p_pod, aspect='equal', cmap='RdBu_r', vmin=-vmax_plot, vmax=vmax_plot,
                        extent=[0, Lx, Lz, 0])
    axes[1].set_ylabel('Depth (m)')
    plt.colorbar(im1, ax=axes[1], label='Pa', fraction=0.046, pad=0.04)
    title1 = axes[1].set_title(f'POD-ROM (q={q_pod})')

    im2 = axes[2].imshow(p_error, aspect='equal', cmap='RdBu_r', vmin=-err_max, vmax=err_max,
                        extent=[0, Lx, Lz, 0])
    axes[2].set_xlabel('Range (m)')
    axes[2].set_ylabel('Depth (m)')
    plt.colorbar(im2, ax=axes[2], label='Pa', fraction=0.046, pad=0.04)
    title2 = axes[2].set_title('Error')

    # Add slider
    ax_slider = plt.axes([0.15, 0.02, 0.55, 0.03])
    slider = Slider(ax_slider, 'Frame', 0, n_frames - 1, valinit=0, valstep=1)

    # Add play/pause button
    ax_button = plt.axes([0.75, 0.02, 0.1, 0.03])
    button = Button(ax_button, 'Play')

    # Animation state
    anim_running = [False]
    current_frame = [0]

    def update(frame_num):
        frame_num = int(frame_num)
        idx = frame_indices[frame_num]
        pod_idx = int(idx * len(t_pod) / len(t_full))
        pod_idx = min(pod_idx, len(t_pod) - 1)
        
        p_full = X_full[N:, idx].reshape(Nx, Nz).T
        p_pod = (Phi @ x_pod[pod_idx, :])[N:].reshape(Nx, Nz).T
        p_error = p_full - p_pod
        
        t_ms = t_full[idx] * 1000
        rel_err = np.linalg.norm(p_error) / (np.linalg.norm(p_full) + 1e-12) * 100
        
        im0.set_data(p_full)
        im1.set_data(p_pod)
        im2.set_data(p_error)
        
        title0.set_text(f'Full-order (t={t_ms:.0f} ms)')
        title1.set_text(f'POD-ROM q={q_pod} (t={t_ms:.0f} ms)')
        title2.set_text(f'Error ({rel_err:.1f}%)')
        
        fig.canvas.draw_idle()

    def on_slider_change(val):
        current_frame[0] = int(val)
        update(val)

    def animate():
        if anim_running[0]:
            current_frame[0] = (current_frame[0] + 1) % n_frames
            slider.set_val(current_frame[0])
            fig.canvas.draw_idle()
            fig.canvas.flush_events()
            timer.start(50)  # 50ms = 20fps

    def on_button_click(event):
        if anim_running[0]:
            anim_running[0] = False
            button.label.set_text('Play')
            timer.stop()
        else:
            anim_running[0] = True
            button.label.set_text('Pause')
            animate()

    # Connect callbacks
    slider.on_changed(on_slider_change)
    button.on_clicked(on_button_click)

    # Timer for animation
    timer = fig.canvas.new_timer(interval=50)
    timer.add_callback(animate)

    plt.show()
else:
    %matplotlib qt

    import matplotlib.pyplot as plt
    from matplotlib.widgets import Slider, Button
    import numpy as np

    # Select frames
    n_frames = 200
    frame_indices = np.linspace(0, len(t_pod) - 1, n_frames, dtype=int)

    N = Nx * Nz

    # Get global colorscale from POD reconstruction
    vmax_global = 0
    for idx in frame_indices[::10]:
        p_sample = (Phi @ x_pod[idx, :])[N:]
        vmax_global = max(vmax_global, np.abs(p_sample).max())

    vmax_plot = vmax_global * 0.5

    # Figure sizing
    domain_aspect = Lx / Lz
    plot_width = 12
    plot_height = plot_width / domain_aspect
    fig_height = plot_height + 1.5  # Single plot + space for slider

    # Create figure
    fig, ax = plt.subplots(1, 1, figsize=(plot_width + 2, fig_height))
    plt.subplots_adjust(bottom=0.2)

    # Initialize plot
    p_pod = (Phi @ x_pod[0, :])[N:].reshape(Nx, Nz).T

    im = ax.imshow(p_pod, aspect='equal', cmap='RdBu_r', vmin=-vmax_plot, vmax=vmax_plot,
                extent=[0, Lx, Lz, 0])
    ax.set_xlabel('Range (m)')
    ax.set_ylabel('Depth (m)')
    plt.colorbar(im, ax=ax, label='Pressure (Pa)', fraction=0.046, pad=0.04)
    title = ax.set_title(f'POD-ROM q={q_pod} (t=0 ms)')

    # Add slider
    ax_slider = plt.axes([0.15, 0.08, 0.55, 0.03])
    slider = Slider(ax_slider, 'Frame', 0, n_frames - 1, valinit=0, valstep=1)

    # Add play/pause button
    ax_button = plt.axes([0.75, 0.08, 0.1, 0.03])
    button = Button(ax_button, 'Play')

    # Animation state
    anim_running = [False]
    current_frame = [0]

    def update(frame_num):
        frame_num = int(frame_num)
        idx = frame_indices[frame_num]
        
        p_pod = (Phi @ x_pod[idx, :])[N:].reshape(Nx, Nz).T
        t_ms = t_pod[idx] * 1000
        
        im.set_data(p_pod)
        title.set_text(f'POD-ROM q={q_pod} (t={t_ms:.0f} ms)')
        
        fig.canvas.draw_idle()

    def on_slider_change(val):
        current_frame[0] = int(val)
        update(val)

    def animate():
        if anim_running[0]:
            current_frame[0] = (current_frame[0] + 1) % n_frames
            slider.set_val(current_frame[0])
            fig.canvas.draw_idle()
            fig.canvas.flush_events()
            timer.start(50)

    def on_button_click(event):
        if anim_running[0]:
            anim_running[0] = False
            button.label.set_text('Play')
            timer.stop()
        else:
            anim_running[0] = True
            button.label.set_text('Pause')
            animate()

    # Connect callbacks
    slider.on_changed(on_slider_change)
    button.on_clicked(on_button_click)

    # Timer for animation
    timer = fig.canvas.new_timer(interval=50)
    timer.add_callback(animate)

    plt.show()

In [ ]:
if SAME_SOURCE:
    fig, ax = plt.subplots(figsize=(12, 5))

    half_idx = len(t_full) 

    for i in range(n_phones):
        y_pod_interp = np.interp(t_full[:half_idx], t_pod, y_pod[:, i])
        
        # Normalize by max signal amplitude (not pointwise)
        signal_max = np.abs(y_full[:half_idx, i]).max()
        abs_error = np.abs(y_full[:half_idx, i] - y_pod_interp)
        
        # Error as % of max signal
        rel_error = abs_error / signal_max * 100
        
        ax.plot(t_full[:half_idx] * 1000, rel_error, lw=1.5, label=f'H{i+1}', alpha=0.8)

    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Error (% of max signal)')
    ax.set_title(f'Normalized Error - POD-ROM q={q_pod}')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

In [ ]:
if SAME_SOURCE:
    import time

    # Get stable q values from previous analysis
    stable_qs = [r['q'] for r in results if r['stable']]

    print(f"Testing {len(stable_qs)} stable q values: {stable_qs}")
    print("="*70)

    # Storage for results
    q_results = []

    for q_test in stable_qs:
        print(f"\nTesting q={q_test}...")
        
        # Build ROM
        Phi_q = U[:, :q_test]
        A_pod_q = Phi_q.T @ (A @ Phi_q)
        B_pod_q = Phi_q.T @ B.toarray()
        C_pod_q = C @ Phi_q
        x0_pod_q = Phi_q.T @ x_start
        
        # Simulate with trapezoidal
        t0 = time.perf_counter()
        t_pod_q, y_pod_q, x_pod_q = simulate_pod_trapezoidal(
            A_pod_q, B_pod_q, C_pod_q, eval_u_scaled, (0, t_sim), dt, x0_pod_q
        )
        sim_time = time.perf_counter() - t0
        
        # Check for blowup
        if np.abs(y_pod_q).max() > 1e10 or np.isnan(y_pod_q).any():
            print(f"  ⚠ Blowup detected, skipping")
            continue
        
        # Compute errors at hydrophones
        errors = []
        for i in range(n_phones):
            y_pod_interp = np.interp(t_full, t_pod_q, y_pod_q[:, i])
            rel_err = np.linalg.norm(y_full[:, i] - y_pod_interp) / (np.linalg.norm(y_full[:, i]) + 1e-12)
            errors.append(rel_err)
        
        avg_error = np.mean(errors) * 100  # Percent
        speedup = full_time / sim_time
        
        q_results.append({
            'q': q_test,
            'time': sim_time,
            'error': avg_error,
            'speedup': speedup,
            'errors': errors
        })
        
        print(f"  Time: {sim_time:.3f}s | Error: {avg_error:.2f}% | Speedup: {speedup:.0f}x")


    # Extract data for plotting
    qs = [r['q'] for r in q_results]
    times = [r['time'] for r in q_results]
    errors = [r['error'] for r in q_results]

    # Single plot with dual y-axes
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Left y-axis: Error
    color1 = 'tab:blue'
    ax1.set_xlabel('Number of modes (q)')
    ax1.set_ylabel('Average error (%)', color=color1)
    ax1.semilogy(qs, errors, 'o-', color=color1, markersize=6, lw=2, label='Error')
    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.axhline(1, color=color1, linestyle='--', alpha=0.5)

    # Right y-axis: Time
    ax2 = ax1.twinx()
    color2 = 'tab:green'
    ax2.set_ylabel('Computation time (s)', color=color2)
    ax2.plot(qs, times, 's-', color=color2, markersize=6, lw=2, label='Time')
    ax2.tick_params(axis='y', labelcolor=color2)

    # Title and grid
    ax1.set_title('POD-ROM: Accuracy vs Computation Time')
    ax1.grid(True, alpha=0.3)

    # Combined legend
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc='center right')

    plt.tight_layout()
    plt.show()